#### Name : Download the dbpedia data into a Azure Blob
#### Author : spaturu
#### Date :  07/31/2017

In [1]:
from azure.storage.blob import BlockBlobService
from azure.storage.blob import ContentSettings
from azure.storage.blob import PublicAccess
import wget
import requests
from os import path

In [2]:
account_name = str(input('Enter UserName for Azure :'))
account_key  = str(input('Enter Key for Azure :'))
container    = str(input('Enter the Container :'))

Enter UserName for Azure :randomtrees
Enter Key for Azure :wvNLlB2cSHhB0OFPRhIQDv+1QBJ1CnwFt+AGfQnL8rTyKTCG90t1Z+aCepe25aol6CKneJYgvHJl5gMtHON7TQ==;EndpointSuffix=core.windows.net
Enter the Container :dbpedia


In [3]:
try:
    block_blob_service = BlockBlobService(account_name = account_name,account_key  = account_key)
except Exception as e:
            print('Please make sure the account name and key are correct.', e)

In [4]:
if container in block_blob_service.list_containers():
    pass
else:
    block_blob_service.create_container(container,public_access=PublicAccess.Container) 

In [5]:
urls = {'wikipedia' :['https://dumps.wikimedia.org/enwiki/20170620/enwiki-20170620-pages-articles-multistream.xml.bz2',
            'https://dumps.wikimedia.org/enwiki/20170620/enwiki-20170620-pages-articles-multistream-index.txt.bz2',
            'https://creativecommons.org/licenses/by-sa/3.0/legalcode',
            'http://www.gnu.org/copyleft/fdl.html'],
        'ontology' :['http://downloads.dbpedia.org/2016-10/dbpedia_2016-10.owl',
           'http://downloads.dbpedia.org/2016-10/dbpedia_2016-10.nt'],
        'datasets' :['http://downloads.dbpedia.org/2016-10/core-i18n/en/'],
        'nlp' :['http://model.dbpedia-spotlight.org/latest_data/en.tar.gz'],
        'dataid':['http://downloads.dbpedia.org/2016-10/2016-10_dataid_catalog.json',
         'http://downloads.dbpedia.org/2016-10/2016-10_dataid_catalog.ttl']}

In [6]:
for blob in urls:
    print(blob)
    for url in urls[blob]:
        print(url)
#         block_blob_service.create_blob_from_path(path.join(container,blob),
#                               wget.download(str(url)),
#                               url.split("/")[-1] ,
#                               content_settings=ContentSettings(content_type="OWL File/owl"))
#         print('uploading file to'+blob+'in a'+container)

wikipedia
https://dumps.wikimedia.org/enwiki/20170620/enwiki-20170620-pages-articles-multistream.xml.bz2
https://dumps.wikimedia.org/enwiki/20170620/enwiki-20170620-pages-articles-multistream-index.txt.bz2
https://creativecommons.org/licenses/by-sa/3.0/legalcode
http://www.gnu.org/copyleft/fdl.html
ontology
http://downloads.dbpedia.org/2016-10/dbpedia_2016-10.owl
http://downloads.dbpedia.org/2016-10/dbpedia_2016-10.nt
datasets
http://downloads.dbpedia.org/2016-10/core-i18n/en/
nlp
http://model.dbpedia-spotlight.org/latest_data/
dataid
http://downloads.dbpedia.org/2016-10/2016-10_dataid_catalog.json
http://downloads.dbpedia.org/2016-10/2016-10_dataid_catalog.ttl
